In [16]:
from components.datasets.dataset_factory import dataset_factory
from components.datasets.dataset_enum import Dataset
from components.text_handler.embedding.embedding_enum import EmbeddingType
from components.text_handler.embedding.embedding_factory import embedding_factory

embedding = embedding_factory(EmbeddingType.GLOVE)
dataset = dataset_factory(Dataset.GO_EMOTIONS, embedding)


.vector_cache/glove.6B.zip: 862MB [13:34, 1.06MB/s]                                
100%|█████████▉| 399999/400000 [00:10<00:00, 36975.01it/s]
